In [1]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from spn.structure.leaves.parametric.Parametric import Categorical, Gaussian
from spn.algorithms.LearningWrappers import learn_parametric
from spn.structure.Base import Context

import pandas as pd
import numpy as np

bunch = fetch_openml(data_id = 181)
bunch.frame

,mcg,gvh,alm,mit,erl,pox,vac,nuc,class_protein_localization
0,0.58,0.61,0.47,0.13,0.5,0.0,0.48,0.22,MIT
1,0.43,0.67,0.48,0.27,0.5,0.0,0.53,0.22,MIT
2,0.64,0.62,0.49,0.15,0.5,0.0,0.53,0.22,MIT
3,0.58,0.44,0.57,0.13,0.5,0.0,0.54,0.22,NUC
4,0.42,0.44,0.48,0.54,0.5,0.0,0.48,0.22,MIT
...,...,...,...,...,...,...,...,...,...
1479,0.81,0.62,0.43,0.17,0.5,0.0,0.53,0.22,ME2
1480,0.47,0.43,0.61,0.40,0.5,0.0,0.48,0.47,NUC
1481,0.67,0.57,0.36,0.19,0.5,0.0,0.56,0.22,ME2
1482,0.43,0.40,0.60,0.16,0.5,0.0,0.53,0.39,NUC


In [2]:
frame = bunch.frame.dropna()
names = frame.columns.tolist()
continuous = 'mcg,gvh,alm,mit,erl,pox,vac,nuc'.split(",")
categories = [name for name in names if name not in continuous]
categorical = [name in categories for name in names]
ptypes = [Categorical if name in categories else Gaussian for name in names]
cols = []
value_text = []
for name in names:
    if name in categories:
        values, text = pd.factorize(frame[name], sort=True)
        value_text.append(text.tolist())
        cols.append(values)
    else:
        cols.append(frame[name])
        value_text.append(None)

data = np.stack(cols, axis=1)

train, test = train_test_split(data, stratify = frame.class_protein_localization, random_state = 0)
print ("%d in train, %d in test" % (len(train), len(test)))

1113 in train, 371 in test


In [3]:
net = learn_parametric(train, 
                       ds_context = Context(parametric_types=ptypes).add_domains(train), 
                       rows = "gmm", ohe = True,
                       min_instances_slice = len(train) / 100)

/home/saurabh/miniconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
/home/saurabh/miniconda3/lib/python3.7/site-packages/sklearn/cross_decomposition/_pls.py:271: UserWarning: Y residual is constant at iteration 0
  warnings.warn(f"Y residual is constant at iteration {k}")
/home/saurabh/miniconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/saurabh/miniconda3/lib/python3.7/site-packages/sklearn/cross_decomposition/_pls.py:271: UserWarning: Y residual is constant at iteration 0
  warnings.warn(f"Y residual is constant at iteration {k}")
/home/saurabh/mi

In [4]:
import csi2
from spn.structure.Base import get_nodes_by_type, Product



def format_condition(condition):
    try:
        a, sign, b = condition.split(" ")
    except ValueError as e:
        raise ValueError("Some nodes don't have conditions. reduce min_impurity_decrease")
    
    if sign in ('==', '!='):
        i = names.index(a)
        v = value_text[i][int(b)]
        return ("%s %s %s" % (a, sign, v))
    else:
        b = float(b)
        return ("%s %s %.2f" % (a, sign, b))

print ('#product nodes = ', len(get_nodes_by_type(net, (Product))))

csi2.annotate_spn(net, names, categorical, 
             min_impurity_decrease = 0.05, max_depth = 2)
rules = csi2.context_specific_independences(net, instance_threshold = 0)


rules = csi2.context_specific_independences(net, instance_threshold = 0)
csis = []
ac = []
cc = []
for i, rule in enumerate(rules):
    antecedent, consequent, *scores = rule
    A = csi2.format_antecedent(antecedent, format_condition)
    C = csi2.format_consequent(consequent)
    a_count = csi2.antecedent_count(A)
    c_count = csi2.consequent_count(C)
    ac.append(a_count)
    cc.append(c_count)
    csis.append ("{%s} => {%s} | %.2f, %.2f, %d | %d %d" % (A, C, *scores, a_count, c_count))
print ("%d, %.2f, %.2f" % (len(csis), np.mean(ac), np.mean(cc)))

rules = csi2.context_specific_independences(net, instance_threshold = len(train) / 20,
                                           precision_threshold = 0.7, recall_threshold = 0.7)
csis = []
ac = []
cc = []
for i, rule in enumerate(rules):
    antecedent, consequent, *scores = rule
    A = csi2.format_antecedent(antecedent, format_condition)
    C = csi2.format_consequent(consequent)
    a_count = csi2.antecedent_count(A)
    c_count = csi2.consequent_count(C)
    ac.append(a_count)
    cc.append(c_count)
    csis.append ("{%s} => {%s} | %.2f, %.2f, %d | %d %d" % (A, C, *scores, a_count, c_count))
print ("%d, %.2f, %.2f" % (len(csis), np.mean(ac), np.mean(cc)))

#product nodes =  181
181, 16.20, 3.26
10, 7.90, 2.30


In [5]:
csis

['{(pox <= 0.25) & (erl <= 0.75)} => {(erl), (pox), (mcg,gvh,alm,mit,nuc,class_protein_localization), (vac)} | 1.00, 1.00, 1091 | 2 4',
 '{[(pox <= 0.25) & (erl <= 0.75)] & [((class_protein_localization != CYT) & (class_protein_localization == NUC)) | (class_protein_localization == CYT)]} => {(mcg,gvh), (alm), (mit,nuc,class_protein_localization)} | 0.89, 0.99, 602 | 5 3',
 '{[(pox <= 0.25) & (erl <= 0.75)] & [((class_protein_localization != CYT) & (class_protein_localization == NUC)) | (class_protein_localization == CYT)] & [(gvh <= 0.48) & (mcg <= 0.50)]} => {(mcg), (gvh)} | 0.89, 0.91, 270 | 7 2',
 '{[(pox <= 0.25) & (erl <= 0.75)] & [((class_protein_localization != CYT) & (class_protein_localization == NUC)) | (class_protein_localization == CYT)] & [class_protein_localization != CYT]} => {(mit,nuc), (class_protein_localization)} | 0.89, 0.99, 289 | 6 2',
 '{[(pox <= 0.25) & (erl <= 0.75)] & [((class_protein_localization != CYT) & (class_protein_localization == NUC)) | (class_protei

In [6]:
from spn.algorithms.Inference import log_likelihood

print ("%.2f" % np.mean(log_likelihood(net, test)))

46.28
